# 15-Minute City Isochrones: Accessibility Analysis

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ucid-foundation/ucid/blob/main/notebooks/02_15min_city_isochrones.ipynb)

---

## Overview

The **15-minute city** concept proposes that urban residents should be able to access all essential services within a 15-minute walk or bike ride. This notebook demonstrates:

1. Understanding the 15-minute city concept
2. Computing walking and cycling isochrones
3. Calculating accessibility scores
4. Identifying service coverage gaps

### Key Metrics

| Category | Amenities | Weight |
|----------|-----------|--------|
| Education | Schools, universities | 20% |
| Healthcare | Hospitals, pharmacies | 20% |
| Commerce | Shops, markets | 15% |
| Recreation | Parks, gyms | 15% |
| Dining | Restaurants, cafes | 15% |
| Services | Banks, post offices | 15% |

---

In [ ]:
# Install dependencies
%pip install -q ucid osmnx networkx

In [ ]:
# Imports
import numpy as np
import pandas as pd

import ucid
from ucid.contexts import FifteenMinContext

print(f"UCID version: {ucid.__version__}")

---

## 1. The 15-Minute City Concept

### 1.1 Background

The 15-minute city (ville du quart d'heure) was popularized by Carlos Moreno and adopted by cities like Paris. The core idea is **chrono-urbanism**: organizing cities around time-based accessibility rather than geographic distance.

### 1.2 Key Principles

1. **Proximity**: Essential services within walking/cycling distance
2. **Diversity**: Mixed-use neighborhoods with varied amenities
3. **Density**: Sufficient population to support local services
4. **Ubiquity**: Equal access across all neighborhoods

In [ ]:
# Initialize the 15MIN context
context = FifteenMinContext()

print("15MIN Context Configuration:")
print(f"  Context ID: {context.context_id}")
print(f"  Walking speed: {context.walking_speed_kmh} km/h")
print(f"  Cycling speed: {context.cycling_speed_kmh} km/h")
print(f"  Max travel time: {context.max_travel_time_min} minutes")

---

## 2. Computing Accessibility Scores

### 2.1 Single Location Analysis

In [ ]:
# Analyze a location in Istanbul
lat, lon = 41.0082, 28.9784  # Sultanahmet

result = context.compute(
    lat=lat,
    lon=lon,
    timestamp="2026W02T14",
)

print("15-Minute City Score:")
print("=" * 40)
print(f"Overall Score:  {result.score}/100")
print(f"Grade:          {result.grade}")
print(f"Confidence:     {result.confidence}%")

In [ ]:
# View component breakdown
print("\nComponent Breakdown:")
print("-" * 40)
for component, score in result.breakdown.items():
    bar = "█" * int(score / 5) + "░" * (20 - int(score / 5))
    print(f"{component:15s}: {bar} {score:.1f}")

### 2.2 Amenity Categories

In [ ]:
# Define amenity categories for 15-minute city
amenity_categories = {
    "education": ["school", "university", "college", "kindergarten"],
    "healthcare": ["hospital", "clinic", "pharmacy", "doctors"],
    "commerce": ["supermarket", "convenience", "marketplace", "mall"],
    "recreation": ["park", "sports_centre", "fitness_centre", "playground"],
    "dining": ["restaurant", "cafe", "fast_food", "bar"],
    "services": ["bank", "post_office", "atm", "library"],
}

print("Amenity Categories:")
for category, amenities in amenity_categories.items():
    print(f"  {category}: {', '.join(amenities)}")

---

## 3. Isochrone Analysis

### 3.1 Walking Isochrone (15 minutes)

In [ ]:
# Calculate walking distance in 15 minutes
walking_speed = 5.0  # km/h
travel_time = 15  # minutes

walking_distance_km = walking_speed * (travel_time / 60)
walking_distance_m = walking_distance_km * 1000

print("Walking Analysis:")
print(f"  Speed: {walking_speed} km/h")
print(f"  Time: {travel_time} minutes")
print(f"  Distance: {walking_distance_m:.0f} meters")
print(f"  Approx. area: {np.pi * walking_distance_km**2:.2f} km²")

### 3.2 Cycling Isochrone (15 minutes)

In [ ]:
# Calculate cycling distance in 15 minutes
cycling_speed = 15.0  # km/h

cycling_distance_km = cycling_speed * (travel_time / 60)
cycling_distance_m = cycling_distance_km * 1000

print("Cycling Analysis:")
print(f"  Speed: {cycling_speed} km/h")
print(f"  Time: {travel_time} minutes")
print(f"  Distance: {cycling_distance_m:.0f} meters")
print(f"  Approx. area: {np.pi * cycling_distance_km**2:.2f} km²")

---

## 4. City-Wide Analysis

### 4.1 Grid-Based Scoring

In [ ]:
# Sample locations across Istanbul
sample_locations = [
    {"name": "Sultanahmet", "lat": 41.0082, "lon": 28.9784},
    {"name": "Taksim", "lat": 41.0370, "lon": 28.9850},
    {"name": "Kadıköy", "lat": 40.9927, "lon": 29.0276},
    {"name": "Beşiktaş", "lat": 41.0428, "lon": 29.0052},
    {"name": "Üsküdar", "lat": 41.0234, "lon": 29.0155},
]

results = []
for loc in sample_locations:
    result = context.compute(lat=loc["lat"], lon=loc["lon"])
    results.append(
        {
            "neighborhood": loc["name"],
            "score": result.score,
            "grade": result.grade,
            "confidence": result.confidence,
        }
    )

df = pd.DataFrame(results)
print("15-Minute City Scores by Neighborhood:")
df.sort_values("score", ascending=False)

### 4.2 Score Distribution

In [ ]:
# Analyze score distribution
print("Score Statistics:")
print(f"  Mean:   {df['score'].mean():.1f}")
print(f"  Median: {df['score'].median():.1f}")
print(f"  Min:    {df['score'].min():.1f}")
print(f"  Max:    {df['score'].max():.1f}")
print(f"  Std:    {df['score'].std():.1f}")

---

## 5. Gap Analysis

### 5.1 Identifying Missing Amenities

In [ ]:
# Analyze gaps in service coverage
def identify_gaps(result):
    """Identify amenity categories with low scores."""
    gaps = []
    for component, score in result.breakdown.items():
        if score < 50:
            gaps.append({"category": component, "score": score})
    return gaps


# Check gaps for each neighborhood
for loc in sample_locations:
    result = context.compute(lat=loc["lat"], lon=loc["lon"])
    gaps = identify_gaps(result)
    if gaps:
        print(f"\n{loc['name']} - Missing Services:")
        for gap in gaps:
            print(f"  - {gap['category']}: {gap['score']:.0f}%")

### 5.2 Improvement Recommendations

In [ ]:
# Generate improvement recommendations
recommendations = {
    "education": "Add local schools or learning centers",
    "healthcare": "Establish clinics or pharmacies",
    "commerce": "Attract shops and markets",
    "recreation": "Create parks and sports facilities",
    "dining": "Support local restaurants and cafes",
    "services": "Add banks and post offices",
}

print("Improvement Recommendations by Category:")
for category, recommendation in recommendations.items():
    print(f"  {category}: {recommendation}")

---

## 6. Temporal Analysis

### 6.1 Time-of-Day Variations

In [ ]:
# Analyze scores at different times of day
hours = [8, 12, 18, 22]
temporal_scores = []

for hour in hours:
    timestamp = f"2026W02T{hour:02d}"
    result = context.compute(lat=41.0082, lon=28.9784, timestamp=timestamp)
    temporal_scores.append({"hour": hour, "score": result.score})

temporal_df = pd.DataFrame(temporal_scores)
print("Score by Time of Day:")
temporal_df

---

## Summary

This notebook demonstrated:

1. **15-Minute City Concept**: Core principles of chrono-urbanism
2. **Accessibility Scoring**: Computing and interpreting 15MIN scores
3. **Isochrone Analysis**: Walking and cycling catchment areas
4. **Gap Analysis**: Identifying and addressing service coverage gaps

### Key Findings

- Central districts typically have higher 15-minute city scores
- Healthcare and education are often the limiting factors
- Cycling significantly expands accessible service areas

---

*Copyright 2026 UCID Foundation. Licensed under EUPL-1.2.*